In [104]:

import numpy as np
import pandas as pd
import mplfinance as mpf
import time
import matplotlib.pyplot as plt
from scipy import io
from utils import *
import random

"""
데이터셋: [[3m*48], [15m*48]]
label: 1 or -1 or 0 << 이후 봉 10개 안에 1%이상 올랐으면 1 내렸으면 -1 이도저도 아니면 0
"""

'\n데이터셋: [*48], *48]]\nlabel: 1 or -1 or 0 << 이후 봉 10개 안에 1%이상 올랐으면 1 내렸으면 -1 이도저도 아니면 0\n'

In [109]:
3*1500*3/60/24

9.375

In [107]:
mat_file_name =  "dataset.mat"
data = io.loadmat(mat_file_name)
inputs = data['inputs']
labels = data['labels']
shuffleidx = [n for n in range(len(inputs))]
random.shuffle(shuffleidx)
testidx = shuffleidx[:10000]

[116258,
 177893,
 194383,
 130596,
 85481,
 17993,
 51446,
 39093,
 263325,
 73686,
 245532,
 246317,
 189429,
 64867,
 235460,
 118817,
 100401,
 187245,
 232370,
 8825,
 94192,
 132403,
 228094,
 178790,
 265411,
 200173,
 241099,
 99840,
 196215,
 117435,
 223275,
 38440,
 173662,
 258956,
 83175,
 46257,
 13435,
 33949,
 44488,
 108043,
 240335,
 192381,
 247866,
 247888,
 38228,
 24229,
 114954,
 15162,
 156738,
 256683,
 127649,
 185723,
 62170,
 89747,
 21181,
 18171,
 180352,
 18387,
 215851,
 55052,
 207970,
 4569,
 11750,
 10734,
 52191,
 249585,
 67038,
 146387,
 215916,
 40784,
 131790,
 123415,
 214864,
 101254,
 229752,
 172021,
 45431,
 19628,
 155537,
 256544,
 31858,
 2298,
 204270,
 49797,
 65585,
 2699,
 131607,
 94270,
 103110,
 244187,
 98256,
 106174,
 174289,
 1689,
 265663,
 161045,
 171826,
 95799,
 148578,
 58530,
 88964,
 251195,
 96015,
 187920,
 209633,
 235280,
 214546,
 229924,
 151014,
 72308,
 52462,
 211088,
 216416,
 63393,
 49362,
 148727,
 158795,


In [108]:
data['inputs'][shuffleidx]

array([[[0.7168, 0.7163, 0.7175, ..., 0.7218, 0.7218, 0.721 ],
        [0.7255, 0.7264, 0.7258, ..., 0.7202, 0.7209, 0.721 ]],

       [[0.7017, 0.7022, 0.7032, ..., 0.7076, 0.7067, 0.7076],
        [0.7129, 0.7129, 0.7129, ..., 0.7065, 0.7079, 0.7076]],

       [[0.7243, 0.7224, 0.723 , ..., 0.7353, 0.7345, 0.7343],
        [0.7311, 0.7375, 0.7455, ..., 0.7274, 0.7333, 0.7343]],

       ...,

       [[0.7202, 0.7218, 0.7222, ..., 0.7128, 0.7128, 0.714 ],
        [0.7266, 0.729 , 0.7339, ..., 0.7187, 0.7155, 0.714 ]],

       [[0.7332, 0.7344, 0.7324, ..., 0.7243, 0.7239, 0.724 ],
        [0.7267, 0.7301, 0.7284, ..., 0.7237, 0.7241, 0.724 ]],

       [[0.7328, 0.7327, 0.7295, ..., 0.7258, 0.726 , 0.7269],
        [0.7379, 0.7346, 0.7338, ..., 0.7255, 0.7264, 0.7269]]])

In [3]:
binance = get_binance()

In [42]:
n = 48
label_n = 10
sym = SYMLIST[0]
df3 = past_data(binance, sym=sym, tf='3m', limit=1500)['close']
df15 = past_data(binance, sym=sym, tf='15m', limit=1500)['close']
print(len(df3))
idx = 1

1500


In [79]:
# input_3 = df3[-(idx+n+label_n):-(idx+label_n)]
# idx15 = (idx+label_n)//5+1
# input_15 = df15[-(idx15+n):-idx15]
# input_15 = input_15.append(input_3[-1:])
# input_3, input_15

In [80]:
# label_vec = df3[-(idx+label_n):-idx]
# future_low, future_high, past = min(label_vec), max(label_vec), label_vec[0]
# diff_ratio_l = (future_low - past)/past*100
# diff_ratio_h = (future_high - past)/past*100
# diff_ratio_l, diff_ratio_h

In [88]:
ref = 0.6

data = pd.DataFrame(columns = ['inputs', 'label'])

dataset = {'inputs': [], 'labels': []}
labelinfo = {"1":0, "-1":0, "0":0}
for sym in SYMLIST:
    for idx in range(1, len(df)-n-label_n):
        input_3 = df3[-(idx+n+label_n):-(idx+label_n)]
        idx15 = (idx+label_n)//5+1
        input_15 = df15[-(idx15+n):-idx15-1]
        input_15 = pd.concat([input_15, input_3[-1:]])

        label_vec = df3[-(idx+label_n):-idx]
        future_low, future_high, past = min(label_vec), max(label_vec), label_vec[0]
        diff_ratio_l = (future_low - past)/past*100
        diff_ratio_h = (future_high - past)/past*100

        label = 0

        if diff_ratio_l > -ref and diff_ratio_h > ref:
                labelinfo['1'] += 1
                label = diff_ratio_h
        elif diff_ratio_l < -ref and diff_ratio_h < ref:
                labelinfo['-1'] += 1
                label = diff_ratio_l
        else:
            labelinfo['0'] += 1

        input_ = np.concatenate([np.expand_dims(input_3, 0), np.expand_dims(input_15, 0)], axis=0)
        dataset['inputs'].append(input_)
        dataset['labels'].append(label)
    print(labelinfo)
print(len(dataset['labels']))
mat_file = io.savemat(mat_file_name, dataset)

{'1': 89, '-1': 165, '0': 1187}
{'1': 178, '-1': 330, '0': 2374}
{'1': 267, '-1': 495, '0': 3561}
{'1': 356, '-1': 660, '0': 4748}
{'1': 445, '-1': 825, '0': 5935}
{'1': 534, '-1': 990, '0': 7122}
{'1': 623, '-1': 1155, '0': 8309}
{'1': 712, '-1': 1320, '0': 9496}
{'1': 801, '-1': 1485, '0': 10683}
{'1': 890, '-1': 1650, '0': 11870}
{'1': 979, '-1': 1815, '0': 13057}
{'1': 1068, '-1': 1980, '0': 14244}
{'1': 1157, '-1': 2145, '0': 15431}
{'1': 1246, '-1': 2310, '0': 16618}
{'1': 1335, '-1': 2475, '0': 17805}
{'1': 1424, '-1': 2640, '0': 18992}
{'1': 1513, '-1': 2805, '0': 20179}
{'1': 1602, '-1': 2970, '0': 21366}
{'1': 1691, '-1': 3135, '0': 22553}
{'1': 1780, '-1': 3300, '0': 23740}
{'1': 1869, '-1': 3465, '0': 24927}
{'1': 1958, '-1': 3630, '0': 26114}
{'1': 2047, '-1': 3795, '0': 27301}
{'1': 2136, '-1': 3960, '0': 28488}
{'1': 2225, '-1': 4125, '0': 29675}
{'1': 2314, '-1': 4290, '0': 30862}
{'1': 2403, '-1': 4455, '0': 32049}
{'1': 2492, '-1': 4620, '0': 33236}
{'1': 2581, '-1': 

In [112]:
import torch
import torch.nn as nn
a = torch.Tensor([1,2,3,4,5])
b = torch.Tensor([1,2,3,4,4])

loss_fn = nn.L1Loss()
loss_fn(a,b)

tensor(0.2000)